**In this notebook I will take a molecular example to create UCC wave opeartor. Then we will show how to factorize it using the cartan decomposition..** 

In [1]:
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
import numpy as np

# Set molecule parameters.
basis = 'dz'
multiplicity = 1
verbose = 4
n_points = 4
bond_length_interval = 1.0 / n_points
bond_length = 0.9

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 1
run_fci = 1
delete_input = True
delete_output = True

# Generate molecule at different bond lengths.
hf_energies = []
fci_energies = []
mp2_energies = []
ccsd_energies = []
bond_lengths = []

#bond_length = float(bond_length)
geometry = [('H', (0., 0., 0.)),('H', (bond_length, 0., 0.)),('H',(0., bond_length, 0.)), ('H', (bond_length, bond_length, 0.))]

#geometry = [('H', (0., 0., 0.)),('H', (bond_length, 0., 0.))]
molecule = MolecularData(geometry, basis, multiplicity,
        description=str(round(bond_length, 2)))
# Run pyscf.
molecule = run_pyscf(molecule,
                    run_scf=run_scf,
                    run_mp2=run_mp2,
                    run_cisd=run_cisd,
                    run_ccsd=run_ccsd,
                    run_fci=run_fci)

t2 = molecule.t2
nocc, nvirt = t2.shape[1:3]
#print(t2)
for i in range(nocc):
    for j in range (nocc):
        for a in range(nvirt):
            for b in range(nvirt):
                if (abs(t2[i,j,a,b]) > 1.0e-4):
                    print('indices', i, j, a, b, t2[i, j, a, b])
            

# Print out some results of calculation.
print('\nAt bond length of {} angstrom, molecular hydrogen has:'.format(
    bond_length))
print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))
print('CCSD energy of {} Hartree.'.format(molecule.ccsd_energy))
print('FCI energy of {} Hartree.'.format(molecule.fci_energy))
print('Nuclear repulsion energy between protons is {} Hartree.'.format(
    molecule.nuclear_repulsion))
for orbital in range(molecule.n_orbitals):
    print('Spatial orbital {} has energy of {} Hartree.'.format(
        orbital, molecule.orbital_energies[orbital]))

System: uname_result(system='Darwin', node='0587285091.wireless.umich.net', release='20.6.0', version='Darwin Kernel Version 20.6.0: Wed Jun 23 00:26:31 PDT 2021; root:xnu-7195.141.2~5/RELEASE_X86_64', machine='x86_64', processor='i386')  Threads 1
Python 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
numpy 1.19.2  scipy 1.7.0
Date: Fri Aug 27 12:29:25 2021
PySCF version 1.7.6
PySCF path  /usr/local/anaconda3/lib/python3.7/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 4
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr
[INPUT]  2 H      0.900000000000   0.000000000000   0.000000000000 AA    1.700753512109   0.000000000000   0.000000000000 Bohr
[INPUT]  3 H      0.000000000000   0.9000000

# Use of singlet adapted amplitudes in UCC
   There are two different ways we can use singlet adapted amplitudes in UCC:
   1. E(i $\rightarrow$ a)  = e<sub>$\downarrow$</sub> + e<sub>$\uparrow$</sub> and E(ij $\rightarrow$ ab) = E (i $\rightarrow$ a) E (j $\rightarrow$ b)

In [2]:
from CQS.methods import *
from CQS.util.IO import tuplesToMatrix
import openfermion
from openfermion import FermionOperator


#reshape t2
t2 = np.reshape(t2, (nocc,nocc,nvirt,nvirt))

#get indices of significant t2

indices = np.argwhere(abs(t2) > 1.e-4)

#generate the strings to combine:
ea_up = []
ea_down = []
eb_up = []
eb_down = []

dea_up = []
dea_down = []
deb_up = []
deb_down = []

Tval = []

for k in range(len(indices)):
    ea_up.append(str((indices[k][2]+nocc)*2)+'^ '+str(indices[k][0]*2))
    eb_up.append(str((indices[k][3]+nocc)*2)+'^ '+str(indices[k][1]*2))
    ea_down.append(str((indices[k][2]+nocc)*2+1)+'^ '+str(indices[k][0]*2+1))
    eb_down.append(str((indices[k][3]+nocc)*2+1)+'^ '+str(indices[k][1]*2+1))

    dea_up.append(str(indices[k][0]*2)+'^ '+ str((indices[k][2]+nocc)*2))
    deb_up.append(str(indices[k][1]*2)+'^ '+str((indices[k][3]+nocc)*2))
    dea_down.append(str(indices[k][0]*2+1)+'^ '+ str((indices[k][2]+nocc)*2+1))
    deb_down.append(str(indices[k][1]*2+1)+'^ '+ str((indices[k][3]+nocc)*2+1))

    #value of amplitudes:
    Tval.append(t2[indices[k][0],indices[k][1],indices[k][2],indices[k][3]])
systemSize = (nocc+nvirt)*2
#define the combination of UCC factors
S = []
for k in range(len(indices)):
    exc_comb_a = FermionOperator(ea_up[k]) + FermionOperator(ea_down[k])
    exc_comb_b = FermionOperator(eb_up[k]) + FermionOperator(eb_down[k])
    dexc_comb_a = FermionOperator(dea_up[k]) + FermionOperator(dea_down[k])
    dexc_comb_b = FermionOperator(deb_up[k]) + FermionOperator(deb_down[k])
    
    S.append(Tval[k]*exc_comb_a*exc_comb_b - Tval[k]*dexc_comb_a*dexc_comb_b) 
    

print(S[0])

0.011701823166679975 [0^ 4 0^ 4] +
0.011701823166679975 [0^ 4 1^ 5] +
0.011701823166679975 [1^ 5 0^ 4] +
0.011701823166679975 [1^ 5 1^ 5] +
-0.011701823166679975 [4^ 0 4^ 0] +
-0.011701823166679975 [4^ 0 5^ 1] +
-0.011701823166679975 [5^ 1 4^ 0] +
-0.011701823166679975 [5^ 1 5^ 1]


In [3]:
#Jordan Wigner Transform
SPauli = openfermion.jordan_wigner(S[0])
print(SPauli)

0.002925455791669994j [X0 X1 X4 Y5] +
0.002925455791669994j [X0 X1 Y4 X5] +
-0.002925455791669994j [X0 Y1 X4 X5] +
0.002925455791669994j [X0 Y1 Y4 Y5] +
-0.002925455791669994j [Y0 X1 X4 X5] +
0.002925455791669994j [Y0 X1 Y4 Y5] +
-0.002925455791669994j [Y0 Y1 X4 Y5] +
-0.002925455791669994j [Y0 Y1 Y4 X5]


In [4]:
#Custom Function to convert OpenFermion operators to a format readable by CQS:
#Feel free to use or modify this code, but it is not built into the CQS package
def OpenFermionToCQS(H, systemSize):
        """
        Converts the Operators to a list of (PauliStrings)

        Args:
            H(obj): The OpenFermion Operator
            systemSize (int): The number of qubits in the system
        """
        stringToTuple  = {   
                        'X': 1,
                        'Y': 2,
                        'Z': 3
                    }
        opList = []
        coList = [] 
        for op in H.terms.keys(): #Pulls the operator out of the QubitOperator format
            coList.append(H.terms[op])
            opIndexList = []
            opTypeDict = {}
            tempTuple = ()
            for (opIndex, opType) in op:
                opIndexList.append(opIndex)
                opTypeDict[opIndex] = opType
            for index in range(systemSize):
                if index in opIndexList:
                    tempTuple += (stringToTuple[opTypeDict[index]],)
                else:
                    tempTuple += (0,)
            opList.append(tempTuple)
        return (coList, opList)
    
#The new format looks like:
print(OpenFermionToCQS(SPauli, systemSize))

([-0.002925455791669994j, 0.002925455791669994j, 0.002925455791669994j, 0.002925455791669994j, -0.002925455791669994j, -0.002925455791669994j, -0.002925455791669994j, 0.002925455791669994j], [(2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (2, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (2, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)])


In [5]:
#Now, we can put all this together:

#Step 1: Create an UCC object
UCC = Hamiltonian(systemSize)

#Use Hamiltonian.addTerms to build the Hubbard model Hamiltonian:
UCC.addTerms(OpenFermionToCQS(SPauli, systemSize))
#This gives:
UCC.getHamiltonian(type='printText')
#There's an IIII term we would rather not deal with, so we can remove it like this:
#HubbardH.removeTerm((0,0,0,0))
#This gives:
#print('Idenity/Global Phase removed:')
#HubbardH.getHamiltonian(type='printText')

#Be careful choosing an involution, because it might now decompose such that the Hamiltonian is in M:
try:
    UCCCartan = Cartan(UCC)
except Exception as e:
    print('Default Even/Odd Involution does not work:')
    print(e)
#print('countY does work through. g = ')
#UCCCartan = Cartan(UCC, involution='countY')
print(UCCCartan.g)




-0.002925455791669994j * YYIIYXIIIIIIIIII
0.002925455791669994j * YXIIYYIIIIIIIIII
0.002925455791669994j * XXIIYXIIIIIIIIII
0.002925455791669994j * XYIIYYIIIIIIIIII
-0.002925455791669994j * YXIIXXIIIIIIIIII
-0.002925455791669994j * YYIIXYIIIIIIIIII
-0.002925455791669994j * XYIIXXIIIIIIIIII
0.002925455791669994j * XXIIXYIIIIIIIIII
[(2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (2, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (2, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)]
